In [1]:
%pip install --upgrade python-dotenv nest_asyncio pydantic google-genai requests pandas pyyaml

from IPython.display import clear_output ; clear_output()

In [2]:
import os
import random
import asyncio

from dotenv import load_dotenv
import nest_asyncio

from textwrap import dedent
from IPython.display import display, Markdown
def md(str): display(Markdown(str))

from pydantic import BaseModel, Field
from enum import Enum

import pandas as pd

import yaml

from google import genai
from google.genai import types

load_dotenv()

nest_asyncio.apply()

_gemini_client_aio = genai.Client(api_key=os.getenv('GEMINI_API_KEY')).aio

G25PRO = 'gemini-2.5-pro-preview-03-25'
G25FLASH = 'gemini-2.5-flash-preview-04-17'

async def gemini(
        prompt,
        pro = False, max_tokens = None, temperature = None,
        budget = None, schema = None):
    config = {}
    if max_tokens is not None:
        config['max_output_tokens'] = max_tokens
    if temperature is not None:
        config['temperature'] = temperature
    if budget is not None:
        config['thinking_config'] = {'thinking_budget': budget}
    if schema is not None:
        config['response_mime_type'] = 'application/json'
        config['response_schema'] = schema
    
    response = await _gemini_client_aio.models.generate_content(
        model=(G25PRO if pro else G25FLASH),
        contents=prompt,
        config=config,
    )
    
    if schema is not None:
        return response.parsed
    else:
        return response.text

In [ ]:
class Location(str, Enum):
    NATURE = 'nature'
    OFFICE = 'office'
    PUB = 'pub'
    STREET = 'street'
    HOME = 'home'

class Mood(str, Enum):
    HAPPY = 'happy'
    SAD = 'sad'
    CALM = 'calm'
    SCARY = 'scary'

class TimeOfDay(str, Enum):
    MORNING = 'morning'
    AFTERNOON = 'afternoon'
    EVENING = 'night'

class Gender(str, Enum):
    MALE = 'male'
    FEMALE = 'female'
    OTHER = 'other'

class Person(BaseModel):
    name: str = Field(description="Name of the person")
    age: int = Field(description="Age of the person")
    gender: Gender = Field(description="Gender of the person")

class Story(BaseModel):
    people: list[Person] = Field(description="List of people in the story")
    location: Location = Field(description="Location of the story")
    time_of_day: TimeOfDay = Field(description="Time of day in the story")
    mood: Mood = Field(description="Mood of the story")

async def create_story():
    num_people = random.randint(2, 4)
    location = random.choice([loc.value for loc in Location])
    mood = random.choice([mood.value for mood in Mood])
    time_of_day = random.choice([time.value for time in TimeOfDay])

    prompt = dedent(f"""
        - Write a very short story depicting a situation.
        - Describe, the people (with their name, age, gender ({', '.join([gender.value for gender in Gender])}).
        - Describe the location, the time of day, and the mood with sufficient detail,
          to allow the reader to discern them clearly and unambiguously. The reader will be tested
          on their ability to identify the people, location, time of day, and mood from the story,
          so be sure to include enough detail for them to do so, and help them out by ensuring that
          the story describes them clearly enough to make them easy to identify.
        - Don't use any special characters or markdown formatting.
        - The story should be roughly 350 tokens long.

        <story-specification>
          - Number of people: {num_people}
          - Location: {location}
          - Time of day: {time_of_day}
          - Mood: {mood}
        </story-specification>
    """)

    response = await gemini(prompt, max_tokens=2500, temperature=0.8, budget=2000)

    return prompt, response

async def parse_story(story):
    prompt = dedent(f"""
        Parse the story and extract the following information:
        - People: name, age, gender ({', '.join([gender.value for gender in Gender])})
        - Location: {', '.join([loc.value for loc in Location])}
        - Time of day: {', '.join([time.value for time in TimeOfDay])}
        - Mood: {', '.join([mood.value for mood in Mood])}

    <story>
      {story}
    </story>
    """)
    response = await gemini(prompt, schema=Story, budget=3000, temperature=0.3)
    response.people = sorted(response.people, key=lambda x: x.name)
    return yaml.dump(
        response.model_dump(mode='json', by_alias=True),
        sort_keys=True, indent=2
    )

In [11]:

prompt, response = await create_story()
parsed = await parse_story(response)

md(f"### Prompt\n\n```{prompt}```\n\n### Response\n\n{response}\n\n### Parsed\n\n```\n{parsed}\n```")

### Prompt

```
- Write a very short story depicting a situation.
- Describe, the people (with their name, age, gender (male, female, other).
- Describe the location, the time of day, and the mood with sufficient detail,
  to allow the reader to discern them clearly and unambiguously. The reader will be tested
  on their ability to identify the people, location, time of day, and mood from the story,
  so be sure to include enough detail for them to do so, and help them out by ensuring that
  the story describes them clearly enough to make them easy to identify.
- Don't use any special characters or markdown formatting.
- The story should be roughly 350 tokens long.

<story-specification>
  - Number of people: 2
  - Location: office
  - Time of day: afternoon
  - Mood: sad
</story-specification>
```

### Response

Sarah, a woman in her late 30s, sat quietly at her desk. Across the small cubicle partition, Mark, a man in his early 40s, looked down at his hands resting on his keyboard. They were in their office, surrounded by other empty desks, humming computers, and stacks of files. The fluorescent lights overhead cast a sterile glow. It was late afternoon; weak sunlight slanted through the blinds on the far window, painting long, pale stripes across the carpeted floor. The usual office bustle had faded, replaced by a heavy silence. Sarah dabbed at her eyes with a tissue, her shoulders slumping slightly. Mark let out a slow, shaky breath, the sound barely audible in the quiet room. The air between them felt thick with shared sadness, a quiet grief hanging heavy in the late day air.

### Parsed

```
location: office
mood: sad
people:
- age: 42
  gender: male
  name: Mark
- age: 38
  gender: female
  name: Sarah
time_of_day: afternoon

```

In [12]:
async def a_gen_item():
    _, response = await create_story()
    parsed = await parse_story(response)
    return response, parsed

stories = pd.DataFrame(
    await asyncio.gather(*[a_gen_item() for _ in range(100)]),
    columns=['input', 'output'],
)

display(stories.head(3).style.set_properties(
    **{'text-align': 'left', 'vertical-align': 'top', 'white-space': 'pre-wrap', 'width': '50%'},
))

,input,output
0,"The campfire crackled softly, a gentle murmur in the profound quiet of the night. Sarah, a woman of 35, sat cross-legged, her eyes reflecting the low flames, a look of peaceful contemplation on her face. Beside her, Mark, a man aged 38, reclined against a fallen log, hands clasped behind his head, utterly absorbed in the vast, star-dusted canvas above. His breathing was deep and relaxed. A few feet away, Lily, an 8-year-old girl, hummed a soft, tuneless song while gently sifting cool earth through her fingers. Her older brother, Tom, a 10-year-old boy, lay flat on his back on a thick blanket, his gaze fixed on the silvery path the moon cast across the smooth, dark surface of the lake just beyond the trees. The air was cool and still, carrying only the clean scents of pine needles, damp soil, and woodsmoke. Aside from the occasional distant hoot of an owl or the almost inaudible lapping of water, the stillness was complete. They were deep in a secluded clearing, surrounded by ancient trees and the quiet presence of the water. A palpable sense of peace and tranquility settled over the small group, each person finding their own quiet space in the communal calm under the expansive night sky.",location: nature mood: calm people: - age: 8 gender: female name: Lily - age: 38 gender: male name: Mark - age: 35 gender: female name: Sarah - age: 10 gender: male name: Tom time_of_day: night
1,"The late afternoon sun cast long, gentle shadows across the quiet workspace. Sarah, a female aged 32, sat at her desk, her gaze fixed on the monitor in front of her. The only sounds were the low hum of the computer equipment and the occasional soft click of a mouse. It was clearly an office environment, with rows of tidy desks, filing cabinets lining one wall, and a whiteboard covered in faded markers. Across the aisle, Mark, a male aged 45, was leaning back in his chair, a half-empty mug of lukewarm coffee beside his keyboard. He was looking out the large window, his expression peaceful and relaxed. The air in the room felt still and calm, a sense of tranquil productivity pervading the space. Neither person spoke, content in the quiet routine of the workday winding down. The light filtering through the blinds indicated the sun was lower in the sky, firmly placing the time in the afternoon. Everything felt unhurried and serene.",location: office mood: calm people: - age: 45 gender: male name: Mark - age: 32 gender: female name: Sarah time_of_day: afternoon
2,"Sarah, a 28 year old female, nervously hugged her arms, the silence of the place pressing in. Beside her, Mark, a 35 year old male, kept glancing towards the darker corners of the room. They were the only two left. The office occupied the entire fifth floor, a sprawling maze of empty cubicles and silent computers under the glow of emergency lights. Desks stretched out into the gloom, monitors reflecting only the dim light. Outside the large windows, the city was a distant sparkle against the absolute black of the night sky. It was well after midnight, the building quiet and deserted. An unsettling stillness hung in the air, broken only by the faint, inexplicable sounds the old building seemed to make in the dark. A cold draft slithered through the space, raising goosebumps on Sarah's arms. Mark shifted his weight, the sound unnaturally loud. They both felt it, the palpable sense of not being alone, of unseen eyes watching them from the shadows that clung to the edges of the empty office. It was an oppressive, chilling feeling.",location: office mood: scary people: - age: 35 gender: male name: Mark - age: 28 gender: female name: Sarah time_of_day: night


In [13]:
stories.to_csv("dataset.csv", index=False)